In [1]:
import os
from dotenv import load_dotenv


load_dotenv("DB_details.env")  

db_config = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT")),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASS"),
    "database": os.getenv("DB_NAME")

}


print("Loaded Database config successfully", db_config)

Loaded Database config successfully {'host': '127.0.0.1', 'port': 3306, 'user': 'root', 'password': '#Eduvos1121', 'database': 'video_store'}


In [13]:
import mysql.connector
from datetime import date

def get_current_vid_id(vname):
    try:
        conn = mysql.connector.connect(**db_config)
        cur = conn.cursor()
        cur.execute("SELECT MIN(videoId) FROM videos WHERE vname = %s", (vname,))
        row = cur.fetchone()
        cur.close()
        conn.close()
        return row[0] if row and row[0] is not None else None
    except Exception as err:
        print("Error (get_current_vid_id):", err)
        return None

def get_next_vid_id():
    try:
        conn = mysql.connector.connect(**db_config)
        cur = conn.cursor()
        cur.execute("SELECT COALESCE(MAX(videoId), 0) + 1 FROM videos")
        next_id = cur.fetchone()[0]
        cur.close()
        conn.close()
        return next_id
    except Exception as err:
        print("Error (get_next_vid_id):", err)
        return None

def get_next_version_of_title(video_id):
    try:
        conn = mysql.connector.connect(**db_config)
        cur = conn.cursor()
        cur.execute("SELECT COALESCE(MAX(videoVer), 0) + 1 FROM videos WHERE videoId = %s", (video_id,))
        next_ver = cur.fetchone()[0]
        cur.close()
        conn.close()
        return next_ver
    except Exception as err:
        print("Error (get_next_version_of_title):", err)
        return None

def movie_registration(vname, vtype):
    vtype = (vtype or "")
    if vtype not in ("R", "B"):
        print("Enter a valid type: 'R' (Red box) or 'B' (Black box).")
        return False

    try:
        # if movie name exists -> same videoId, bump version; else new id, version = 1
        video_id = get_current_vid_id(vname)
        if video_id is None:
            video_id = get_next_vid_id()
            if video_id is None:
                print("Could not determine next videoId.")
                return False
            video_ver = 1
        else:
            video_ver = get_next_version_of_title(video_id)
            if video_ver is None:
                print("Could not determine next version for existing title.")
                return False

        conn = mysql.connector.connect(**db_config)
        cur = conn.cursor()
        cur.execute("""
            INSERT INTO videos (videoId, videoVer, vname, `type`, dateAdded)
            VALUES (%s, %s, %s, %s, %s)
        """, (video_id, video_ver, vname, vtype, date.today()))
        conn.commit()
        cur.close()
        conn.close()

        print(f"Movie registered: {vname}, videoId={video_id}, version={video_ver}")
        return True
    except Exception as err:
        print("Error (movie_registration):", err)
        return False
